In [2]:
import requests
import random
import time
import socket
import http.client
import json
import pandas as pd

In [239]:
def get_html(index=0, minP=None, maxP=None, minBed=None, maxBed=None):
    base_url = 'https://www.rightmove.co.uk/api/_search?'
    payload = {
        "index": index*24,
        "sortType": 2, #大于1000条可通过排序正反抓取两次
        "numberOfPropertiesPerPage": 24,
        "locationIdentifier": "REGION^79192",
        "viewType": "LIST",
        "includeLetAgreed": "true",
        "minBedrooms": minBed,
        "maxBedrooms": maxBed,
        "minPrice": minP,
        "maxPrice": maxP,
        "propertyTypes": ["flat", "detached", "semi-detached", "terraced", "bungalow"],
        "must_have": [],
        "dontShow": ["student", "retirement"],
        "channel": "RENT",
        "areaSizeUnit":"sqft",
        "currencyCode":"GBP",
        "isFetching":"false"
        }
    header = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en,en-US;q=0.8,zh-CN;q=0.7,zh;q=0.5,zh-TW;q=0.3,zh-HK;q=0.2',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'
            }
    timeout = random.choice(range(80, 100))
    while True:
        try:
            response = requests.get(base_url, params=payload, headers = header, timeout = timeout)
            response.encoding = 'utf-8'
            break
        except socket.timeout as e:
            print(e)
            time.sleep(random.choice(range(20, 60)))
        except socket.error as e:
            print(e)
            time.sleep(random.choice(range(0, 60)))
        except http.client.BadStatusLine as e:
            print(e)
            time.sleep(random.choice(range(30, 60)))
        except http.client.IncompleteRead as e:
            print(e)
            time.sleep(random.choice(range(20, 60)))
    print(response.url)
    return response.json()

def data_output(data, filename):
    with open(filename, 'a', errors='ignore', newline='', encoding='utf-8') as f:
        f_csv = csv.writer(f)
        f_csv.writerows(data)
    
if __name__ == '__main__':
   # pro_df = pd.DataFrame()
    mylist1 = [None, 500, 600]
    mylist2 = [600, 700, 800, 900, 1000]
    mylist3 = [1000, 1100, 1250, None]
    for j in range(len(mylist2)-1):
        html = get_html(index=0, minP=mylist2[j], maxP=mylist2[j+1])
        page = html['pagination']['total']
        for i in range(page):
            html = get_html(index=i, minP=mylist2[j], maxP=mylist2[j+1])
            pro_df = pro_df.append(pd.DataFrame(html['properties']))
            time.sleep(random.choice(range(2)))

  #  data_output(result, 'rightmove.csv')

https://www.rightmove.co.uk/api/_search?index=0&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=0&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=24&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTyp

https://www.rightmove.co.uk/api/_search?index=480&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=504&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=528&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&proper

https://www.rightmove.co.uk/api/_search?index=984&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=600&maxPrice=700&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=0&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=0&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTy

https://www.rightmove.co.uk/api/_search?index=456&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=480&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=504&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&proper

https://www.rightmove.co.uk/api/_search?index=960&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=984&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=700&maxPrice=800&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=0&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&property

https://www.rightmove.co.uk/api/_search?index=432&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=456&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=480&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&proper

https://www.rightmove.co.uk/api/_search?index=936&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=960&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=984&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=800&maxPrice=900&proper

https://www.rightmove.co.uk/api/_search?index=408&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=432&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=456&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&pro

https://www.rightmove.co.uk/api/_search?index=912&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=936&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&propertyTypes=flat&propertyTypes=detached&propertyTypes=semi-detached&propertyTypes=terraced&propertyTypes=bungalow&dontShow=student&dontShow=retirement&channel=RENT&areaSizeUnit=sqft&currencyCode=GBP&isFetching=false
https://www.rightmove.co.uk/api/_search?index=960&sortType=2&numberOfPropertiesPerPage=24&locationIdentifier=REGION%5E79192&viewType=LIST&includeLetAgreed=true&minPrice=900&maxPrice=1000&pro

In [197]:
    for j in range(len(mylist3)-1): #mylist1 and mylist3
        html = get_html(index=0, minP=mylist3[j], maxP=mylist3[j+1])
        page = html['pagination']['total']
        for i in range(page):
            html = get_html(index=i, minP=mylist3[j], maxP=mylist3[j+1])
            pro_df = pro_df.append(pd.DataFrame(html['properties']))
            time.sleep(random.choice(range(1, 2)))

0
1


In [255]:
pro_df.columns.tolist()

['addedOrReduced',
 'auction',
 'bedrooms',
 'channel',
 'commercial',
 'contactUrl',
 'countryCode',
 'customer',
 'development',
 'displayAddress',
 'displaySize',
 'displayStatus',
 'distance',
 'enhancedListing',
 'featuredProperty',
 'feesApply',
 'feesApplyText',
 'firstVisibleDate',
 'formattedBranchName',
 'formattedDistance',
 'hasBrandPlus',
 'heading',
 'hidden',
 'id',
 'isRecent',
 'keywordMatchType',
 'keywords',
 'listingUpdate',
 'location',
 'numberOfFloorplans',
 'numberOfImages',
 'numberOfVirtualTours',
 'onlineViewingsAvailable',
 'premiumListing',
 'price',
 'productLabel',
 'propertyImages',
 'propertySubType',
 'propertyTypeFullDescription',
 'propertyUrl',
 'residential',
 'saved',
 'showOnMap',
 'staticMapUrl',
 'students',
 'summary',
 'transactionType']

##### 重置行索引

In [240]:
#pro_df = pro_df.reset_index()
pro_df

,addedOrReduced,auction,bedrooms,channel,commercial,contactUrl,countryCode,customer,development,displayAddress,...,propertySubType,propertyTypeFullDescription,propertyUrl,residential,saved,showOnMap,staticMapUrl,students,summary,transactionType
0,Reduced on 23/10/2020,False,1,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 63855, 'brandPlusLogoURI': '/64k/...",False,"Vimto Gardens, Chapel Street, Salford, Lancash...",...,Apartment,1 bedroom apartment,/property-to-rent/property-73110580.html,True,False,True,None,False,***************AVAILABLE IMMEDIATLEY**********...,rent
1,Added on 09/10/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 79669, 'brandPlusLogoURI': '/80k/...",False,"Newark Road, Rochdale",...,Semi-Detached,3 bedroom semi-detached house,/property-to-rent/property-98175257.html,True,False,True,None,False,"AVAILABLE TO VIEW SOON, SORRY NO PETS, FACE MA...",rent
2,Added on 21/10/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 56965, 'brandPlusLogoURI': '/comp...",False,"Staley Road, Mossley, OL5",...,Terraced,3 bedroom terraced house,/property-to-rent/property-98770490.html,True,False,True,None,False,*VIRTUAL VIEWING AVAILABLE* Three bedroom end ...,rent
3,Added on 06/10/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 66312, 'brandPlusLogoURI': '/comp...",False,"Coniston Avenue, Manchester, M9",...,Terraced,2 bedroom terraced house,/property-to-rent/property-71764875.html,True,False,True,None,False,A beautifully refurbished 2 bed terrace house ...,rent
4,Added on 21/08/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 3242, 'brandPlusLogoURI': '/brand...",False,"Smallbrook Lane, Leigh WN7",...,House,3 bedroom house,/property-to-rent/property-72684852.html,True,False,True,None,False,Now available to rent is this attractive three...,rent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,Added on 07/10/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 14294, 'brandPlusLogoURI': '/comp...",False,"The Danube, City Road East, River City, M15 4TH",...,Apartment,2 bedroom apartment,/property-to-rent/property-73959228.html,True,None,True,None,False,Contemporary apartment in excellent location j...,rent
20,Reduced on 27/10/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 85297, 'brandPlusLogoURI': '/comp...",False,"City Loft, 94 The Quays, Salford, Lancashire, M50",...,Apartment,2 bedroom apartment,/property-to-rent/property-65905045.html,True,None,True,None,False,CITY LOFTS - Available Now- No Parking 2 doubl...,rent
21,Added on 04/11/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 66830, 'brandPlusLogoURI': '/bran...",False,"City South, City Centre, M15",...,Apartment,2 bedroom apartment,/property-to-rent/property-99466796.html,True,None,True,None,False,Fancy living in the heart of Manchester city c...,rent
22,Added on 13/10/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 183263, 'brandPlusLogoURI': '/com...",False,"Northern Angel, 15 Dyche Street, Red Bank",...,Apartment,2 bedroom apartment,/property-to-rent/property-98839220.html,True,None,True,None,False,The Northern Angel development is perfectly lo...,rent


##### 查找重复

In [193]:
dups = pro_df.pivot_table(index=['id'], aggfunc='size')
#dups = pd.DataFrame(dups)
dups[dups==2]

id
50810189    2
69952719    2
74686002    2
75154365    2
86366470    2
86371705    2
86372506    2
86583325    2
86583454    2
98372018    2
dtype: int64

In [195]:
pro_df.index[pro_df['id']==50810189,].tolist()


[65, 275]

In [100]:
pro_df.loc[pro_df['identifier'].isin(ddd.index[:])] 

,index,address,autoEmailReasonType,bedrooms,branch,commercial,dateShortlisted,development,displayPrices,exactLocationAvailable,...,shouldShowPrice,showLettingFeesMessage,showMap,showStreetView,sortDate,status,summary,transactionTypeId,updateDate,visible
33,8,"Hale Road, Hale Barns",price_reduced,6,"{'identifier': 15554, 'name': 'Hale', 'brandNa...",False,None,False,"[{'displayPrice': '£2,450,000', 'displayPriceQ...",False,...,True,False,True,True,1526895700000,None,"A stunning, architecturally designed, detached...",1,1603890736000,True
47,22,"North Road, Hale",new,NaN,"{'identifier': 15554, 'name': 'Hale', 'brandNa...",True,None,False,"[{'displayPrice': '£2,000,000', 'displayPriceQ...",False,...,True,False,True,True,1464265058000,None,A rare opportunity to acquire a GENEROUS SINGL...,1,1603890736000,True
153,3,"Ringlows Lane, Rochdale",price_reduced,5,"{'identifier': 59264, 'name': 'Knutsford', 'br...",False,None,False,"[{'displayPrice': '£1,250,000', 'displayPriceQ...",False,...,True,False,True,True,1470745718000,None,Discerning purchasers searching for quality an...,1,1601480742000,True
311,11,"Groby Road, Bowdon",new,NaN,"{'identifier': 15554, 'name': 'Hale', 'brandNa...",True,None,False,"[{'displayPrice': '£895,000', 'displayPriceQua...",False,...,True,False,True,True,1520005245000,None,An ideal opportunity to purchase a private sin...,1,1603890736000,True
618,18,"Rivershill Gardens, Hale Barns",price_reduced,4,"{'identifier': 15554, 'name': 'Hale', 'brandNa...",False,None,False,"[{'displayPrice': '£680,000', 'displayPriceQua...",False,...,True,False,True,True,1560250868000,None,"A stunning, individually designed, recently re...",1,1603890736000,True


##### 去重

In [258]:
pro_df_dedup = pro_df.drop_duplicates("id")
pro_df_dedup

,addedOrReduced,auction,bedrooms,channel,commercial,contactUrl,countryCode,customer,development,displayAddress,...,propertySubType,propertyTypeFullDescription,propertyUrl,residential,saved,showOnMap,staticMapUrl,students,summary,transactionType
0,Reduced on 23/10/2020,False,1,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 63855, 'brandPlusLogoURI': '/64k/...",False,"Vimto Gardens, Chapel Street, Salford, Lancash...",...,Apartment,1 bedroom apartment,/property-to-rent/property-73110580.html,True,False,True,None,False,***************AVAILABLE IMMEDIATLEY**********...,rent
1,Added on 09/10/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 79669, 'brandPlusLogoURI': '/80k/...",False,"Newark Road, Rochdale",...,Semi-Detached,3 bedroom semi-detached house,/property-to-rent/property-98175257.html,True,False,True,None,False,"AVAILABLE TO VIEW SOON, SORRY NO PETS, FACE MA...",rent
2,Added on 21/10/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 56965, 'brandPlusLogoURI': '/comp...",False,"Staley Road, Mossley, OL5",...,Terraced,3 bedroom terraced house,/property-to-rent/property-98770490.html,True,False,True,None,False,*VIRTUAL VIEWING AVAILABLE* Three bedroom end ...,rent
3,Added on 06/10/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 66312, 'brandPlusLogoURI': '/comp...",False,"Coniston Avenue, Manchester, M9",...,Terraced,2 bedroom terraced house,/property-to-rent/property-71764875.html,True,False,True,None,False,A beautifully refurbished 2 bed terrace house ...,rent
4,Added on 21/08/2020,False,3,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 3242, 'brandPlusLogoURI': '/brand...",False,"Smallbrook Lane, Leigh WN7",...,House,3 bedroom house,/property-to-rent/property-72684852.html,True,False,True,None,False,Now available to rent is this attractive three...,rent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Added on 18/09/2020,False,4,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 32438, 'brandPlusLogoURI': '/comp...",False,"Oak Avenue, Hindley Green",...,Cottage,4 bedroom cottage,/property-to-rent/property-85987015.html,True,False,True,None,False,Borron Shaw are delighted to offer To Let this...,rent
5,Added on 04/11/2020,False,2,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 45816, 'brandPlusLogoURI': '/46k/...",False,"Munday Street, Ancoats, Manchester, M4",...,Apartment,2 bedroom apartment,/property-to-rent/property-99428036.html,True,False,True,None,False,***AVAILABLE 03-12-2020** - VITALSPACE ESTATE ...,rent
6,Added on 23/10/2020,False,1,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 78006, 'brandPlusLogoURI': '/79k/...",False,"Princess Street, Manchester",...,Apartment,1 bedroom apartment,/property-to-rent/property-86262835.html,True,False,True,None,False,"*AVAILABLE NOW* A true Manchester gem, minutes...",rent
7,Reduced on 08/10/2020,False,1,RENT,False,/property-to-rent/contactBranch.html?propertyI...,GB,"{'branchId': 68309, 'brandPlusLogoURI': '/69k/...",False,"St. Marys Parsonage, Manchester, M3",...,Flat,1 bedroom flat,/property-to-rent/property-84932782.html,True,False,True,None,False,** AVAILABLE NOW ** ****360 ONLINE VIRTUAL VIE...,rent


In [242]:
pro_df_dedup.to_csv('rightmove1.csv')